## 考试文件，需要请下载download到本地，作答完成后，再上传到云平台students文件夹内即可。

### 1、用jieba分词库和停用词表对下面句子进行分词并去除停用词？（停用词表可以用课上用的）

In [1]:
import jieba
import re
import pandas as pd
sentence = " \u3000\u30002016年是综   艺井喷的一年，《2016年中国网络视听发展研究报告》数据显示，截至2016年11月30日，视频网站备案的网络综艺为618档，6637期。2016年可谓是当之无愧的网络综艺黄金年。“爆款”网综《奇葩说》的第三季继续延续了火热，《火星情报局》、《拜托了冰箱》等网络综艺也都获得了较高的点击率与不俗的口碑，然而这些网络综艺距“现象级”都还有一定的差距。 "

In [13]:
seg = jieba.lcut(sentence)
stopwords=pd.read_csv("NLP_project/data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
result = [i for i in seg if i not in stopwords.stopword.tolist()]

In [14]:
result

[' ',
 '综',
 ' ',
 ' ',
 ' ',
 '艺',
 '井喷',
 '一年',
 '中国',
 '网络',
 '视听',
 '发展',
 '研究',
 '报告',
 '数据',
 '显示',
 '视频',
 '网站',
 '备案',
 '网络',
 '综艺',
 '618',
 '档',
 '6637',
 '期',
 '可谓',
 '当之无愧',
 '网络',
 '综艺',
 '黄金',
 '爆款',
 '网综',
 '奇葩',
 '说',
 '第三季',
 '延续',
 '火热',
 '火星',
 '情报局',
 '拜托',
 '冰箱',
 '网络',
 '综艺',
 '高',
 '点击率',
 '不俗',
 '口碑',
 '网络',
 '综艺',
 '现象',
 '级',
 '差距',
 ' ']

### 2、计算TFIDF值？
写下TFIDF计算公式，并手动或调库计算单词 "first" 的TFIDF值。

In [16]:
corpus = ['this is the first document.',
          'this is the  second document.',
          'and the third one.',
          'first the third document?']

答案：

TF-IDF(Term Frequency-Inverse Document Frequency, 词频-逆文件频率)

$TF-IDF=TF∗IDF$

$TF_W = \frac{在某一类中词条w出现的次数}{该类中所有的词条数目}$

$IDF = log(\frac{语料库的文档总数}{包含词条w的文档数+1})$


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
tfidf_matrix = tfidf_vec.fit_transform(corpus)
print(tfidf_vec.get_feature_names())
print(tfidf_vec.vocabulary_)
num = tfidf_vec.vocabulary_['first']
#print(tfidf_matrix.toarray())
print("first in 'this is the first document.': ",tfidf_matrix.toarray()[0][num])
print("first in 'first the third document?': ",tfidf_matrix.toarray()[3][num])

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
{'this': 8, 'is': 3, 'the': 6, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 7, 'one': 4}
first in 'this is the first document.':  0.49425506218038123
first in 'first the third document?':  0.5685556582078485


### 3、朴素贝叶斯完成中文文本分类课程中，除了贝叶斯和SVM，请换一个模型完成，代码和测试结果粘贴在下面？

In [36]:
import jieba
import pandas as pd
df_technology = pd.read_csv("NLP_project/data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("NLP_project/data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("NLP_project/data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("NLP_project/data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("NLP_project/data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [37]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception:
            print (line)
            continue 

#生成训练数据
sentences = []

preprocess_text(technology, sentences, 'technology')
preprocess_text(car, sentences, 'car')
preprocess_text(entertainment, sentences, 'entertainment')
preprocess_text(military, sentences, 'military')
preprocess_text(sports, sentences, 'sports')

In [38]:
import random
random.shuffle(sentences)
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

In [39]:
len(x_train)

65696

In [42]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

class TextClassifier():

    def __init__(self, classifier=RandomForestClassifier(n_estimators=8)):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)



In [43]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.8088040549796794


### 4、Fasttext相比于word2vec做了什么改进？（面试高频）

答案：

改善运算效率：fastText 模型使用了层次 Softmax 技巧。层次 Softmax 技巧建立在哈弗曼编码的基础上，对标签进行编码，能够极大地缩小模型预测目标的数量。fastText 也利用了类别（class）不均衡这个事实（一些类别出现次数比其他的更多），通过使用 Huffman 算法建立用于表征类别的树形结构。因此，频繁出现类别的树形结构的深度要比不频繁出现类别的树形结构的深度要小，这也使得进一步的计算效率更高。

fasttext有别于word2vec的另一点是加了ngram切分，将长词再通过ngram切分为几个短词，这样对于未登录词也可以通过切出来的ngram词向量合并为一个词。

### 5、对于用CNN类模型作文本分类和RNN类模型做文本分类，你觉得各自的优势和特点是什么，分别适合什么场景？（面试高频）

答案：cnn善于捕捉局部信息，RNN擅长处理时序信息

